<a href="https://colab.research.google.com/github/gretiere545/tradunion/blob/master/corpus_sync_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synchronisation des Corpus ASAMLA

## Liste des Corpus à sauvegarder

In [ ]:
!pip install gspread-formatting
import pandas as pd
import numpy as np
import uuid
import random
import os
import re
from gspread_formatting import *
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
!export PYTHONIOENCODING=utf8
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [ ]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

# Chargement des Corpus
#### Ouverture de la Sheet Centrale (corpus_central_base)
###### Cette feuille est la référence et ne peut être modifiée que par l'administrateur

In [ ]:
# Ouverture de la Sheet Centrale (corpus_central_base)
uri = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(uri)
t_corpus = wb_central.worksheet('med_vac_synthese')
data_t_corpus = t_corpus.get_all_values()
df_central = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
df_central = df_central[['uid','expression','glossaire','état','date','commentaires','index']]

### Traitement des corpus de travail (fichier unique à onglets)

In [ ]:
vk_lang_dict=[
{'language':'Arabe','trigramme':'ams','uri':'https://docs.google.com/spreadsheets/d/1HPc_VrpG0hNfgXaQB2WgeLJMsUstNAFJbAIa1xNE1HM'},
{'language':'Turc','trigramme':'tur','uri':'https://docs.google.com/spreadsheets/d/1QTWWnOqLy8GNHfgv-8GN_4e0uxcS266dtlFR4Cjy7AY'},
{'language':'Russe','trigramme':'rus','uri':'https://docs.google.com/spreadsheets/d/193hOJRFSXJ5QOsApEGoZdUgqTCIdlQKQ6wbXboJhmL8'},
{'language':'Ukrainien','trigramme':'ukr','uri':'https://docs.google.com/spreadsheets/d/1PIRf2L6T_RHkcfWBtEI-IRhqHw5e0z-IdpNqqqlx-wI'},
{'language':'Roumain','trigramme':'rou','uri':'https://docs.google.com/spreadsheets/d/1NlA186MFJVuZVHrz8ITSbgXsF8f7UkVxU54we2XAnUA'},
{'language':'Hongrois','trigramme':'hun','uri':'https://docs.google.com/spreadsheets/d/1EZ-o6I-r1ixFCLcPcEJRT_eaSJn0Du18DWrU3QTgejE'},
{'language':'Tigrinya','trigramme':'tig','uri':'https://docs.google.com/spreadsheets/d/1m778VcSgqu02vHZrMOOWfLUIMVPDYmQNhz1X3myU4-A'},
{'language':'Albanais','trigramme':'alb','uri':'https://docs.google.com/spreadsheets/d/1sU7xGhHfxqkzKm4y2EwHqiZhxc2yjuOMLGDdFXcQkNg'},
{'language':'Géorgien','trigramme':'geo','uri':'https://docs.google.com/spreadsheets/d/1PH58PFAfIItNr7AJpjiFp1yo6bMlgWy8C6P245FBWIA'},
{'language':'Arménien','trigramme':'arm','uri':'https://docs.google.com/spreadsheets/d/1kUXerjG8EpagYaDHKBMteGriCXOUHRz0YgoyKXINU9g'},
{'language':'Dari','trigramme':'dar','uri':'https://docs.google.com/spreadsheets/d/1QOAUccnSLIv5qqxkjq0w-tPOX2L0pqMGgB3EqhosOEc'},
{'language':'Pashto','trigramme':'pst','uri':'https://docs.google.com/spreadsheets/d/1_5h6U8g9ymzeX9jjGFeBOancOYSMolsNG0LV0xzk4W8'},
{'language':'Fârsi','trigramme':'prs','uri':'https://docs.google.com/spreadsheets/d/1PlitiSFEqF3KKzDiojIv_XMMKxvQfREPIJjmm8drFQE'},
{'language':'Azéri','trigramme':'aze','uri':'https://docs.google.com/spreadsheets/d/1Er7Y22UsH2CkROYi0Pbdm1H5hYU8JTnWhYRIG3Syh7E'},
{'language':'Espagnol','trigramme':'esp','uri':'https://docs.google.com/spreadsheets/d/1AGHS6d1ujAgbasHtK4_no4wH2B8UoQQgh4UJnPtZ3fk'},
{'language':'Amharique','trigramme':'amh','uri':'https://docs.google.com/spreadsheets/d/1Oi90BvCMzsawUfFohpzWDKblUA0CdTKt1UIFJVCP088'},
{'language':'Peul','trigramme':'fma','uri':'https://docs.google.com/spreadsheets/d/1He3466RGbwkMIJUbo0eHY6lVnr7ka7t-K0qp_LL5r1M'},
]

In [ ]:
# Corpus de travail
uri = "https://docs.google.com/spreadsheets/d/1CclzYfFCW4srA3Lq_np2LpSrxj84JpcbzytL449DH8E"
wb_travail = gc.open_by_url(uri)

## Chargement des corpus dans un DF (un par langue/onglet)

In [ ]:
def fetch_onglet_into_df(wb, idx):
  #ouverture de l'onglet Corpus dans un DF
  t_corpus = wb.get_worksheet(idx+1)
  data_t_corpus = t_corpus.get_all_values()
  df = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
  return df

In [ ]:
df_corpus=[] 
#ouverture de chaque onglet Corpus de travail dans un DF
for index, item in enumerate(vk_lang_dict):
  df_corpus.append (fetch_onglet_into_df(wb_travail,index))

0 {'language': 'Arabe', 'trigramme': 'ams', 'uri': 'https://docs.google.com/spreadsheets/d/1HPc_VrpG0hNfgXaQB2WgeLJMsUstNAFJbAIa1xNE1HM'}
1 {'language': 'Turc', 'trigramme': 'tur', 'uri': 'https://docs.google.com/spreadsheets/d/1QTWWnOqLy8GNHfgv-8GN_4e0uxcS266dtlFR4Cjy7AY'}
2 {'language': 'Russe', 'trigramme': 'rus', 'uri': 'https://docs.google.com/spreadsheets/d/193hOJRFSXJ5QOsApEGoZdUgqTCIdlQKQ6wbXboJhmL8'}
3 {'language': 'Ukrainien', 'trigramme': 'ukr', 'uri': 'https://docs.google.com/spreadsheets/d/1PIRf2L6T_RHkcfWBtEI-IRhqHw5e0z-IdpNqqqlx-wI'}
4 {'language': 'Roumain', 'trigramme': 'rou', 'uri': 'https://docs.google.com/spreadsheets/d/1NlA186MFJVuZVHrz8ITSbgXsF8f7UkVxU54we2XAnUA'}
5 {'language': 'Hongrois', 'trigramme': 'hun', 'uri': 'https://docs.google.com/spreadsheets/d/1EZ-o6I-r1ixFCLcPcEJRT_eaSJn0Du18DWrU3QTgejE'}
6 {'language': 'Tigrinya', 'trigramme': 'tig', 'uri': 'https://docs.google.com/spreadsheets/d/1m778VcSgqu02vHZrMOOWfLUIMVPDYmQNhz1X3myU4-A'}
7 {'language': 'Albanai

### Recopie dans le DF central de chaque DF de travail : 1 DF = 1 colonne

In [ ]:
for index, item in enumerate(vk_lang_dict):
  df_central = pd.merge(df_central, df_corpus[index][['uid','traduction']],on='uid', how='left')
  df_central.rename({'traduction':item['trigramme']}, axis=1, inplace=True)



### Formatage de la feuille centrale

In [ ]:
def format_feuille(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)
  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:G', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=7)
  set_column_width(ws, 'A', 100)
  set_column_width(ws, 'B', 300)
  set_column_width(ws, 'H:', 200)

## Sauvegarde de la feuille centrale

In [ ]:
df_central_synth = df_central.copy()
nom_onglet = 'med_vac_synthese'
# si l'onglet existe déjà
try:
  ws = wb_central.worksheet(nom_onglet)
  wb_central.del_worksheet(ws)
except:
  print ("Onglet inexistant !")
# supprime les doublons
df_central_synth = df_central_synth.drop_duplicates()
wb_central.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb_central.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_central_synth)
format_feuille(wb_central, nom_onglet)

#FIN